In [89]:
import pandas as pd
from pathlib import Path
import os
import numpy as np

from IPython.display import display


pd.set_option("display.precision", 13)

In [92]:
# location of data 
root = Path('D:\\')
dir = 'D:/Kinematic_data'

csv = 'C:/Users/petra/REU/unix_videos.csv'
dfv = pd.read_csv(csv)

# find all kinematics files in the folder
#file_list = list(dir.glob('*.txt'))
#file_list

In [147]:
8 * np.ones(4, dtype=np.uint8)

array([8, 8, 8, 8], dtype=uint8)

In [150]:

# loop over videos
for index, row in dfv.iterrows():
    file = int(row["video_id"])
    fname = f'''{file}.txt'''
    print (fname)

    filename = f'''{dir}/{fname}'''
    print(filename)
    filepath = Path(filename)

    # Get all 88 column names (will look like "sensor_6_azimuth")
    col_names = ["name", "status", "x", "y", "z", "azimuth", "elevation", "roll", "button", "quality", "time"]
    cols = []
    for sensor_num in range(1,9):
        for col_name in col_names:
            cols.append(f"sensor_{sensor_num}_{col_name}")

    # Read all sensor data into one dataframe
    df = pd.read_csv(filename, sep='\s+', header=0, names=cols)

    # Separate out each sensor into its own dataframe
    sensor_dfs = []
    for sensor_num in range(1,9):
        sensor_cols = [f"sensor_{sensor_num}_{col_name}" for col_name in col_names]
        # Rename ("sensor_6_azimuth" -> "azimuth") since sensor name isn't needed anymore
        sensor_df = df[sensor_cols].rename({sc: c for sc,c in zip(sensor_cols, col_names)}, axis=1)
        sensor_dfs.append(sensor_df)
        #print(sensor_df)
        

    fps = 30
    #frames = 50

    # useful time interval
    # loop over videos - iterate through the unix start and end times for tasks
    start_timestamp = row['unix_start_time']
    end_timestamp = row['unix_end_time']

    # total number of frames
    frames = int(np.floor((end_timestamp - start_timestamp) * fps))
    #display(frames)

    # new dataframe for the time of each frame
    f = np.arange(frames) # array of frame numbers 0,1,2...,frames-1
    t = start_timestamp + f / fps
    times = pd.DataFrame({'time': t, 'frame': f})
    print(times)
    if False:
        display(times)

    merged_dfs = []
    for sensor_index, sensor_df in enumerate(sensor_dfs):
        sensor_num = sensor_index + 1
        print(sensor_df)

        # test time intervals
        assert times.time.values[0] > sensor_df.time.values[0], 'starting violation'
        assert times.time.values[-1] < sensor_df.time.values[-1], 'ending violation'

        #join the motion data and the frame times data frames 
        #merged_df = pd.merge(times, df[['t', 'x']], left_on='time', right_on='t', how='outer')

        merged_df = times.merge(sensor_df, how='outer', on='time')
        display(merged_df)


        # sort the unix times column in numerical order 
        merged_df = merged_df.sort_values(by=['time'])
        merged_df.set_index('time', inplace=True)
        display(merged_df)

        # sensor columns to interpolate are all but a few
        sensor_cols = list(sensor_df.columns) # everything kinematic file
        for c in ['status', 'time']: # excluding these columns
            sensor_cols.remove(c)
        print (sensor_cols)
        
        # interpolate selected columns in place
        #col_inter = merged_df['x', 'y', 'z', 'azimuth', 'elevation', 'roll', 'button', 'quality']

        for c in sensor_cols:
            merged_df[c].interpolate(inplace=True, method='index')
        #merged_df.interpolate(inplace=True, method='index')
        
        # exclude rows without frame numbers, i.e., NaN 
        #merged_df = merged_df[merged_df.frame.apply(lambda f: f >= 0)]

        merged_df = merged_df[merged_df['frame'].notna()]

        merged_df.insert(0, 'sensor_num', sensor_num * np.ones(len(merged_df), dtype=np.uint8))

        display(merged_df)

        merged_dfs.append(merged_df)

        # Save the DataFrame to a CSV file
        video = int(row['video_id'])
        output = Path(f'''{video}_{sensor_num}.csv''')
        # delete output csv if already exists
        if output.exists():
            output.unlink() 
        merged_df.to_csv(output, index=False)
    # end of loop over sensors
    # ???
    all_df = pd.concat(merged_dfs)
    display(all_df)
# end of loop over videos



891038.txt
D:/Kinematic_data/891038.txt
                      time  frame
0      1.6662120631000e+09      0
1      1.6662120631333e+09      1
2      1.6662120631667e+09      2
3      1.6662120632000e+09      3
4      1.6662120632333e+09      4
...                    ...    ...
22497  1.6662128130000e+09  22497
22498  1.6662128130333e+09  22498
22499  1.6662128130667e+09  22499
22500  1.6662128131000e+09  22500
22501  1.6662128131333e+09  22501

[22502 rows x 2 columns]
           name  status        x        y        z  azimuth  elevation  \
0      Sensor1:  0x0000  476.176 -181.384 -359.085  -81.712    -40.645   
1      Sensor1:  0x0000  477.962 -183.728 -364.889  -82.008    -42.335   
2      Sensor1:  0x0000  479.301 -187.077 -371.140  -82.362    -43.858   
3      Sensor1:  0x0000  481.087 -190.091 -376.275  -82.761    -45.371   
4      Sensor1:  0x0000  483.877 -191.877 -381.409  -82.999    -46.752   
...         ...     ...      ...      ...      ...      ...        ...   
90337  S

,time,frame,name,status,x,y,z,azimuth,elevation,roll,button,quality
0,1.6662120631000e+09,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.6662120631333e+09,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.6662120631667e+09,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.6662120632000e+09,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.6662120632333e+09,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
112387,1.6662129131910e+09,NaN,Sensor1:,0x1001,914.288,-400.831,23.552,170.795,-14.962,-150.419,0.0,16.0
112388,1.6662129132010e+09,NaN,Sensor1:,0x1001,914.288,-400.831,27.794,170.683,-15.005,-151.242,0.0,16.0
112389,1.6662129132110e+09,NaN,Sensor1:,0x1001,914.288,-398.599,28.910,170.752,-15.433,-152.101,0.0,16.0
112390,1.6662129132210e+09,NaN,Sensor1:,0x1001,914.288,-394.246,32.928,171.111,-15.636,-153.035,0.0,17.0


,frame,name,status,x,y,z,azimuth,elevation,roll,button,quality
time,,,,,,,,,,,
1.6662111630840e+09,NaN,Sensor1:,0x0000,476.176,-181.384,-359.085,-81.712,-40.645,-85.866,0.0,3.0
1.6662111630940e+09,NaN,Sensor1:,0x0000,477.962,-183.728,-364.889,-82.008,-42.335,-84.152,0.0,3.0
1.6662111631040e+09,NaN,Sensor1:,0x0000,479.301,-187.077,-371.140,-82.362,-43.858,-82.715,0.0,3.0
1.6662111631140e+09,NaN,Sensor1:,0x0000,481.087,-190.091,-376.275,-82.761,-45.371,-81.102,0.0,3.0
1.6662111631240e+09,NaN,Sensor1:,0x0000,483.877,-191.877,-381.409,-82.999,-46.752,-79.457,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...
1.6662129131910e+09,NaN,Sensor1:,0x1001,914.288,-400.831,23.552,170.795,-14.962,-150.419,0.0,16.0
1.6662129132010e+09,NaN,Sensor1:,0x1001,914.288,-400.831,27.794,170.683,-15.005,-151.242,0.0,16.0
1.6662129132110e+09,NaN,Sensor1:,0x1001,914.288,-398.599,28.910,170.752,-15.433,-152.101,0.0,16.0


['name', 'x', 'y', 'z', 'azimuth', 'elevation', 'roll', 'button', 'quality']


,sensor_num,frame,name,status,x,y,z,azimuth,elevation,roll,button,quality
time,,,,,,,,,,,,
1.6662120631000e+09,1,0.0,NaN,NaN,914.288000000000,-288.3841978637675,-120.9752042533915,-47.8262035858188,-24.1979980926495,134.2385973106359,0.0,12.0000000000000
1.6662120631333e+09,1,1.0,NaN,NaN,914.288000000000,-287.1708900889302,-119.6204212860310,-47.2757381207830,-24.6590618219965,134.5606654268889,0.0,11.0000000000000
1.6662120631667e+09,1,2.0,NaN,NaN,914.288000000000,-281.1960383377441,-111.4948809098062,-46.3881383305915,-26.1698522280237,135.0957284886632,0.0,10.7333524068379
1.6662120632000e+09,1,3.0,NaN,NaN,914.288000000000,-277.1776063705505,-100.4588425720621,-45.8092013923658,-28.6147860191212,135.7323983024581,0.0,10.0000000000000
1.6662120632333e+09,1,4.0,NaN,NaN,914.288000000000,-277.8026682640727,-97.0063994707102,-45.6483331664402,-30.8487364280094,135.2499319791145,0.0,10.0000000000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1.6662128130000e+09,1,22497.0,NaN,NaN,393.688000000000,-502.0720000000000,-47.8183994707102,66.0360000953675,-65.7624000190735,52.9671999380111,0.0,4.0000000000000
1.6662128130333e+09,1,22498.0,NaN,NaN,393.688000000000,-502.2875338673915,-47.5510000000000,66.0904000429154,-65.7365999570846,52.8931332522709,0.0,4.0000000000000
1.6662128130667e+09,1,22499.0,NaN,NaN,393.576000000000,-502.2950000000000,-47.4091327992752,66.0656665712991,-65.7322666714350,52.9098000858308,0.0,4.0000000000000


...

,time,frame,name,status,x,y,z,azimuth,elevation,roll,button,quality
0,1.6662120631000e+09,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.6662120631333e+09,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.6662120631667e+09,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.6662120632000e+09,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.6662120632333e+09,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
112386,1.6662129131050e+09,NaN,Sensor8:,0x0000,677.540,265.658,60.722,8.716,-2.575,99.170,0.0,20.0
112387,1.6662129131150e+09,NaN,Sensor8:,0x0000,677.540,265.658,60.722,8.723,-2.582,99.170,0.0,20.0
112388,1.6662129131250e+09,NaN,Sensor8:,0x0000,677.428,265.770,60.722,8.730,-2.589,99.170,0.0,20.0
112389,1.6662129131350e+09,NaN,Sensor8:,0x0000,677.428,265.770,60.833,8.737,-2.596,99.177,0.0,20.0


,frame,name,status,x,y,z,azimuth,elevation,roll,button,quality
time,,,,,,,,,,,
1.6662111630050e+09,NaN,Sensor8:,0x0000,683.568,218.331,41.970,3.696,-0.322,97.188,0.0,9.0
1.6662111630150e+09,NaN,Sensor8:,0x0000,683.568,218.331,41.970,3.696,-0.322,97.188,0.0,9.0
1.6662111630250e+09,NaN,Sensor8:,0x0000,683.568,218.331,41.858,3.696,-0.315,97.188,0.0,9.0
1.6662111630350e+09,NaN,Sensor8:,0x0000,683.679,218.442,41.858,3.696,-0.315,97.181,0.0,9.0
1.6662111630450e+09,NaN,Sensor8:,0x0000,683.679,218.442,41.858,3.689,-0.308,97.181,0.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...
1.6662129131050e+09,NaN,Sensor8:,0x0000,677.540,265.658,60.722,8.716,-2.575,99.170,0.0,20.0
1.6662129131150e+09,NaN,Sensor8:,0x0000,677.540,265.658,60.722,8.723,-2.582,99.170,0.0,20.0
1.6662129131250e+09,NaN,Sensor8:,0x0000,677.428,265.770,60.722,8.730,-2.589,99.170,0.0,20.0


['name', 'x', 'y', 'z', 'azimuth', 'elevation', 'roll', 'button', 'quality']


,sensor_num,frame,name,status,x,y,z,azimuth,elevation,roll,button,quality
time,,,,,,,,,,,,
1.6662120631000e+09,8,0.0,NaN,NaN,707.6655535775782,245.5660000000000,54.8060000000000,0.8757779014014,-1.5950000000000,96.9020000000000,0.0,11.0
1.6662120631333e+09,8,1.0,NaN,NaN,707.7890000000000,245.5660000000000,54.9180000000000,0.8593667834919,-1.5880000000000,96.9020000000000,0.0,11.0
1.6662120631667e+09,8,2.0,NaN,NaN,707.8524647974632,245.5660000000000,54.9180000000000,0.8430334501585,-1.5810000000000,96.9020000000000,0.0,11.0
1.6662120632000e+09,8,3.0,NaN,NaN,707.9010000000000,245.5660000000000,54.8060000000000,0.8337001168252,-1.5677001168252,96.8950000000000,0.0,11.0
1.6662120632333e+09,8,4.0,NaN,NaN,708.0389007939347,245.5400992060654,54.8060000000000,0.8243666165987,-1.5513666165987,96.8880000000000,0.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1.6662128130000e+09,8,22497.0,NaN,NaN,679.8840000000000,263.7610000000000,55.2520000000000,7.8925999570846,-1.9100000000000,98.8960000000000,0.0,17.0
1.6662128130333e+09,8,22498.0,NaN,NaN,679.8840000000000,263.7610000000000,55.2520000000000,7.8980000000000,-1.9100000000000,98.8960000000000,0.0,17.0
1.6662128130667e+09,8,22499.0,NaN,NaN,679.8840000000000,263.7610000000000,55.2520000000000,7.8920000000000,-1.9030000000000,98.8882333404859,0.0,17.0


,sensor_num,frame,name,status,x,y,z,azimuth,elevation,roll,button,quality
time,,,,,,,,,,,,
1.6662120631000e+09,1,0.0,NaN,NaN,914.288,-288.3841978637675,-120.9752042533915,-47.8262035858188,-24.1979980926495,134.2385973106359,0.0,12.0000000000000
1.6662120631333e+09,1,1.0,NaN,NaN,914.288,-287.1708900889302,-119.6204212860310,-47.2757381207830,-24.6590618219965,134.5606654268889,0.0,11.0000000000000
1.6662120631667e+09,1,2.0,NaN,NaN,914.288,-281.1960383377441,-111.4948809098062,-46.3881383305915,-26.1698522280237,135.0957284886632,0.0,10.7333524068379
1.6662120632000e+09,1,3.0,NaN,NaN,914.288,-277.1776063705505,-100.4588425720621,-45.8092013923658,-28.6147860191212,135.7323983024581,0.0,10.0000000000000
1.6662120632333e+09,1,4.0,NaN,NaN,914.288,-277.8026682640727,-97.0063994707102,-45.6483331664402,-30.8487364280094,135.2499319791145,0.0,10.0000000000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1.6662128130000e+09,8,22497.0,NaN,NaN,679.884,263.7610000000000,55.2520000000000,7.8925999570846,-1.9100000000000,98.8960000000000,0.0,17.0000000000000
1.6662128130333e+09,8,22498.0,NaN,NaN,679.884,263.7610000000000,55.2520000000000,7.8980000000000,-1.9100000000000,98.8960000000000,0.0,17.0000000000000
1.6662128130667e+09,8,22499.0,NaN,NaN,679.884,263.7610000000000,55.2520000000000,7.8920000000000,-1.9030000000000,98.8882333404859,0.0,17.0000000000000


def interpolation():
    #parser = argparse.ArgumentParser(
        #description='Interolation of motion data on video frames')

    #parser.add_argument('csv_unix', help='File of Start and End UNIX timestamps for processed videos')       
    #parser.add_argument('-f', '--fps', default=30, help="Frame rate")

    #args = parser.parse_args()
    #csv = args.csv_unix

    csv = 'C:/Users/petra/REU/unix_videos.csv'
    dfv = pd.read_csv(csv)

    # convenience class like a dict
    #if args is None:
        #args.Bunch()
        #args.fps = 30
        #args.frames = 50
    fps = 30
    #frames = 50

    # loop over videos - iterate through the unix start and end times for tasks
    for index, row in dfv.iterrows():
        # useful time interval
        start_timestamp = row['unix_start_time']
        end_timestamp = row['unix_end_time']
        
        # total number of frames
      
        frames = int(np.floor((end_timestamp - start_timestamp) * fps))
        #display(frames)
        
        # new dataframe for the time of each frame
        f = np.arange(frames) # array of frame numbers 0,1,2...,frames-1
        t = start_timestamp + f / fps
        times = pd.DataFrame({'time': t, 'frame': f})

        if False:
            display(times)
            display(df[['t', 'x']])

        # test time intervals
        assert times.time.values[0] > sensor_df.time.values[0], 'starting violation'
        assert times.time.values[-1] < sensor_df.time.values[-1], 'ending violation'

        #join the motion data and the frame times data frames 
        #merged_df = pd.merge(times, df[['t', 'x']], left_on='time', right_on='t', how='outer')

        merged_df = times.merge(sensor_df, how='outer', on='time')
        display(merged_df[['time','x','y']])

        # sort the unix times column in numerical order 
        merged_df = merged_df.sort_values('time')
        merged_df.set_index('time', inplace=True)

        # sensor columns to interpolate are all but a few
        sensor_cols = list(sensor_df.columns) # everything kinematic file
        for c in []: # excluding these columns
            sensor_cols.remove(c)
        print (sensor_cols)

        # interpolate selected columns in place 
        for c in sensor_cols:
            merged_df[c].interpolate(inplace=True, method='linear')
        
        display(merged_df)

interpolation()